In [ ]:
cd ../src

In [ ]:
import numpy as np
import os
import pandas as pd

from matplotlib import pyplot as plt

from config import \
    RAND_TEMPERATURES, RAND_COOLING_RATES, \
    RAND_CONST_CITY_COUNT, RAND_CONST_CITY_DIST, \
    RAND_CONST_COOLING_RATE, RAND_CONST_TEMPERATURE

plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [ ]:
results = []
for f in os.listdir("../results"):
    if f.endswith(".csv") and f.startswith("rand"):
        df = pd.read_csv(f"../results/{f}", header=0)
        results.append(df)
df = pd.concat(results)
df.head()
cr_df = df.loc[df["temperature"] == RAND_CONST_TEMPERATURE]
t_df = df.loc[df["cooling_rate"] == RAND_CONST_COOLING_RATE]
print(RAND_CONST_COOLING_RATE)

In [ ]:
city_counts = sorted(cr_df["city_count"].unique())
print(f"{city_counts = }")

# Plot the highest temperature
cr_dists = []
for cr in RAND_COOLING_RATES:
    curr_cr_df = cr_df.loc[cr_df["cooling_rate"] == cr]
    results = []
    for city_count in city_counts:
        dist = curr_cr_df.loc[curr_cr_df["city_count"] == city_count]["solver_dist"]
        results.append(dist.mean())
    cr_dists.append(np.array(results))


for i, dist in enumerate(cr_dists):
    plt.plot(city_counts, dist, label=f"Cooling rate: {RAND_COOLING_RATES[i]}")

plt.xlabel("City Count")
plt.ylabel("Total Distance")
plt.legend()
plt.savefig("../report/images/rand_distance_city_count.jpg")
plt.show()

In [ ]:
# Plot the highest temperature
cr_iterations = []
for cr in RAND_COOLING_RATES:
    curr_cr_df = cr_df.loc[cr_df["cooling_rate"] == cr]
    results = []
    for city_count in city_counts:
        iterations = curr_cr_df.loc[curr_cr_df["city_count"] == city_count]["iterations"]
        results.append(iterations.mean())
    cr_iterations.append(np.array(results))


for i, avg_iteration in enumerate(cr_iterations):
    plt.plot(city_counts, avg_iteration, label=f"Cooling rate: {RAND_COOLING_RATES[i]}")

plt.xlabel("City Count")
plt.ylabel("Average Iterations")
plt.yscale("log")
plt.legend()
# plt.savefig("../report/images/rand_iterations_city_count.jpg")
plt.show()

In [ ]:
#  Show optimality for cooling rate
for city_count in city_counts:
    city_df = cr_df.loc[cr_df["city_count"] == city_count]
    cr_dfs = [city_df.loc[city_df["cooling_rate"] == cr] for cr in RAND_COOLING_RATES]
    dists = [cr_df["solver_dist"] for cr_df in cr_dfs]
    print([dist.std() / dist.mean() for dist in dists])

    plt.boxplot(dists, labels=RAND_COOLING_RATES)
    plt.xlabel("Cooling Rate")
    plt.ylabel("Distance (lower is better)")
    plt.title(f"City Count = {city_count}")
    # plt.savefig(f"../report/images/rand_dist_{city_count}_cr.jpg")
    plt.show()

In [ ]:
avg_city_dists = sorted(t_df["avg_city_dist"].unique())
print(f"{avg_city_dists = }")

# Plot the highest temperature
t_dists = []
for t in RAND_TEMPERATURES:
    curr_t_df = t_df.loc[t_df["temperature"] == t]
    results = []
    for avg_city_dist in avg_city_dists:
        dist = curr_t_df.loc[curr_t_df["avg_city_dist"] == avg_city_dist]["solver_dist"]
        results.append(dist.mean() / avg_city_dist)
    t_dists.append(np.array(results))


for i, dist in enumerate(t_dists):
    plt.plot(avg_city_dists, dist, label=f"Temperature: {RAND_TEMPERATURES[i]}")

plt.xlabel("Average City Distance")
plt.ylabel("Total Distance (Lower is better)")
plt.xscale("log")
plt.legend()
plt.savefig("../report/images/rand_distance_city_dist.jpg")
plt.show()

In [ ]:
avg_city_dists = sorted(t_df["avg_city_dist"].unique())
print(f"{avg_city_dists = }")

# Plot the highest temperature
t_dists = []
for t in RAND_TEMPERATURES:
    curr_t_df = t_df.loc[t_df["temperature"] == t]
    results = []
    for avg_city_dist in avg_city_dists:
        dist = curr_t_df.loc[curr_t_df["avg_city_dist"] == avg_city_dist]["iterations"]
        results.append(dist.mean())
    t_dists.append(np.array(results))


for i, dist in enumerate(t_dists):
    plt.plot(avg_city_dists, dist, label=f"Temperature: {RAND_TEMPERATURES[i]}")

plt.xlabel("Average City Distance")
plt.ylabel("Iterations (Lower is better)")
plt.xscale("log")
plt.legend()
plt.savefig("../report/images/rand_distance_city_dist.jpg")
plt.show()